In [3]:
import torch
from src.CLEAN.utils import *
from src.CLEAN.model import LayerNormNet
import os

esm_embed_dir = '/home/stef/hiec/data/embeddings/esm1b/'

# Redefine format esm with idx 32
def format_esm(a):
    if type(a) == dict:
        a = a['mean_representations'][32]
    return a

def load_esm(fn, dir=esm_embed_dir):
    esm = format_esm(torch.load(dir + fn))
    return esm.unsqueeze(0) # Stick a 1 out in front of shape tuple

ModuleNotFoundError: No module named 'pandas'

In [2]:
# Get CLEAN's embeddings of Eric's precomputed ESM1b

ds = 1
esm_embeds = [load_esm(elt) for elt in os.listdir(esm_embed_dir)[::ds]] # Load Eric's esm
fns = os.listdir(esm_embed_dir)[::ds] # Keep file names


def get_clean_embeds(train_str, esm_embeds):
    # Torch params
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")
    dtype = torch.float32
    
    # load checkpoints
    # NOTE: change this to LayerNormNet(512, 256, device, dtype) 
    # and rebuild with [python build.py install]
    # if inferencing on model trained with supconH loss
    model = LayerNormNet(512, 128, device, dtype)
    checkpoint = torch.load('./data/pretrained/'+ train_str +'.pth', map_location=device) 
    model.load_state_dict(checkpoint)
    model.eval()
        
    # Forward pass
    esm_embeds = torch.cat(esm_embeds).to(device=device, dtype=dtype)
    model_emb = model(esm_embeds)
    return model_emb

clean_embeds = get_clean_embeds("split100", esm_embeds)
clean_embeds.shape, len(fns)


torch.Size([111234, 128])

In [3]:
# Save embeddings

# You must clone a slice of the tensor to
# save data only from that slice, otherwise you
# save the whole tensor each time because 
# actually slicing returns a view, not a new object

# Detach from loss function or whetever to just 
# get the vector
save_to = '/home/stef/hiec/data/embeddings/clean/'
for i in range(clean_embeds.shape[0]):
        torch.save(clean_embeds[i].clone().detach(), save_to + fns[i])

In [ ]:
# Get CLEAN's own embeddings

# def clean_embedding(train_data, test_data, pretrained=True, model_name=None):
#     use_cuda = torch.cuda.is_available()
#     device = torch.device("cuda:0" if use_cuda else "cpu")
#     dtype = torch.float32
#     id_ec_train, ec_id_dict_train = get_ec_id_dict('./data/' + train_data + '.csv')
#     id_ec_test, _ = get_ec_id_dict('./data/' + test_data + '.csv')
#     # load checkpoints
#     # NOTE: change this to LayerNormNet(512, 256, device, dtype) 
#     # and rebuild with [python build.py install]
#     # if inferencing on model trained with supconH loss
#     model = LayerNormNet(512, 128, device, dtype)
    
#     if pretrained:
#         try:
#             checkpoint = torch.load('./data/pretrained/'+ train_data +'.pth', map_location=device)
#         except FileNotFoundError as error:
#             raise Exception('No pretrained weights for this training data')
#     else:
#         try:
#             checkpoint = torch.load('./data/model/'+ model_name +'.pth', map_location=device)
#         except FileNotFoundError as error:
#             raise Exception('No model found!')
        
#     model.load_state_dict(checkpoint)
#     model.eval()
#     # load precomputed EC cluster center embeddings if possible
#     if train_data == "split70":
#         emb_train = torch.load('./data/pretrained/70.pt', map_location=device)
#     elif train_data == "split100":
#         emb_train = torch.load('./data/pretrained/100.pt', map_location=device)
#     else:
#         emb_train = model(esm_embedding(ec_id_dict_train, device, dtype))
        
#     emb_test = model_embedding_test(id_ec_test, model, device, dtype)
#     return emb_test, emb_train, id_ec_train, device

# emb, emb_arr, id_ec, device = clean_embedding("split100", "test", pretrained=True)
# print(emb.shape, emb_arr.shape, len(id_ec))
# foo = torch.load('/home/stef/hiec/CLEAN/app/data/pretrained/100.pt', map_location=device)